In [ ]:
!pip install pyanp
#from IPython.display import display
import pandas as pd
import numpy as np
from pyanp import limitmatrix as lm
#from pandas.io.formats.style import Styler

#Listado de criterios

criteria = {'Closeness gap value': ['Empty travel of MHE flow (inter-functional)','Information flow (inter-functional)',
                                    'Loaded travel of MHE flow (inter-functional)','Loaded travel of MHE flow (intra-functional)',
                                    'Other equipments flow (inter-functional)','Personnel flow (inter-functional)'],
            'Expansion flexibility': ['Ease of expansion','Number of free space Locations with good shape factor',
                                      'Number of free space Locations with usable area','Total area of free space'],
            'Routing flexibility': ['Accessibility of alternate routes','Average number of alternate routes',
                                    'Material handling flexibility'],
            'Volume flexibility': ['Variations demand volume','Variations in MHC'],
            'Productive area utilization':['Non-value adding area','Value adding area'],
            'Human issues': ['Ease of supervision and control','Safety and housekeeping']
            }


#Los criterios anteriormente mencionados, pero en una lista
all_criteria = list(criteria.values())
all_criteria = [all_criteria[i][j] for i in range(len(all_criteria)) for j in range(len(all_criteria[i]))]
print(all_criteria)

                                  #Cluster Matrix
cluster_matrix = pd.read_excel('data.xlsx',sheet_name='Cluster Matrix',index_col=0) #Cluster matrix
display(cluster_matrix.style.set_caption('Cluster Matrix'))

                                  #Unweighted Matrix

unweighted = pd.read_excel('data.xlsx',sheet_name='Unweighted Matrix',index_col=0) #Unweighted Matrix
display(unweighted.style.set_caption('Unweighted Matrix'))


                                  #Weighted Matrix
weighted = unweighted.copy() #Copia de la unweighted matrix


#Multiplicamos cada valor de la cluster matrix por su correspondiente cluster en la unweighted matrix
#Para así dar resultado a la weighted matrix
for key1 in criteria:
    for key2 in criteria:
        weighted.loc[criteria[key1],criteria[key2]] = unweighted.loc[criteria[key1],criteria[key2]]*cluster_matrix[key2][key1]

display(weighted.style.set_caption('Weighted Matrix'))



                                  #Limit Matrix
limit_matrix = weighted.to_numpy() #Convertimos a arreglo de numpy
limit_matrix = lm.calculus(limit_matrix) #Aplicamos el algoritmo par calcular la matriz limite
limit_matrix = pd.DataFrame(limit_matrix, index=all_criteria, columns=all_criteria) #Matriz limite a DataFrame
display(limit_matrix.style.set_caption('Limit Matrix'))

                                #Matriz de prioridades para ANP
priority_ANP = pd.DataFrame(columns=['Totals','Priorities']+ all_criteria,index = ['Limit priorities',
                                                                                   'Facility 1',
                                                                                    'Facility 2',
                                                                                  'Facility 3'])

table7 = pd.read_excel('data.xlsx',sheet_name='Table 7',index_col = 0) #Tabla con caracteristicas de las facilities


priority_ANP.iloc[0,2:] = limit_matrix.iloc[:,0]  

minimize_values = ['Non-value adding area']


#Rellenando la tabla con los datos obtenidos de la tabla de caracteristicas de las alternativas
for facility in range(1,4):
    for criterio in all_criteria:
        facility_value = table7.loc[criterio,'Facility '+str(facility)]
        if facility_value > 1 and criterio not in minimize_values:
            maximo = max(table7.loc[criterio,:])
            priority_ANP.loc['Facility '+str(facility),criterio] = facility_value/maximo
        elif facility_value > 1 and criterio in minimize_values:
            minimo = min(table7.loc[criterio,:])
            priority_ANP.loc['Facility '+str(facility),criterio] = minimo/facility_value
        elif facility_value <= 1:
            minimo = min(table7.loc[criterio,:])
            priority_ANP.loc['Facility '+str(facility),criterio] = minimo/facility_value
      

#Valores que no se entrega información y son dados por expertos
priority_ANP.iloc[1:,8] = [0.874,0.382,1] #Ease of expansion
priority_ANP.iloc[1:,19] = [0.303,1,0.55] #Ease of supervision and control
priority_ANP.iloc[1:,20] = [0.303,0.55,1] #Safety and housekeeping
priority_ANP.iloc[1:,12] = [0.630,0.794,1] #Accesibility of alternate routes
priority_ANP.iloc[1:,13] = [1,1,1] #Average number of alternate routes
priority_ANP.iloc[1:,14] = [1,1,1] #Material handling flexibility
priority_ANP.iloc[1:,16] = [0.961,1,0.962] #Variations in MHC


#Totals
total = 0
for fila in range(1,4):
    for columna in range(2,len(all_criteria)+2):
        total += priority_ANP.iloc[fila,columna]*priority_ANP.iloc[0,columna]
    priority_ANP.iloc[fila,0] = total
    total = 0

#Final priorities
for i in range(1,4):
    priority_ANP.iloc[i,1] = priority_ANP.iloc[i,0]/priority_ANP.iloc[:,0].sum()

display(priority_ANP.style.set_caption('Priority Matrix for ANP '))



     |████████████████████████████████| 51kB 4.4MB/s 
['Empty travel of MHE flow (inter-functional)', 'Information flow (inter-functional)', 'Loaded travel of MHE flow (inter-functional)', 'Loaded travel of MHE flow (intra-functional)', 'Other equipments flow (inter-functional)', 'Personnel flow (inter-functional)', 'Ease of expansion', 'Number of free space Locations with good shape factor', 'Number of free space Locations with usable area', 'Total area of free space', 'Accessibility of alternate routes', 'Average number of alternate routes', 'Material handling flexibility', 'Variations demand volume', 'Variations in MHC', 'Non-value adding area', 'Value adding area', 'Ease of supervision and control', 'Safety and housekeeping']


,Closeness gap value,Expansion flexibility,Routing flexibility,Volume flexibility,Productive area utilization,Human issues
Closeness gap value,0.556689,0.075252,0.216149,0.284770,0.167123,0.351592
Expansion flexibility,0.000000,0.313864,0.135323,0.284770,0.252833,0.150278
Routing flexibility,0.236977,0.222785,0.379754,0.166535,0.104679,0.084525
Volume flexibility,0.147901,0.119233,0.047712,0.166535,0.078433,0.219128
Productive area utilization,0.000000,0.186980,0.135323,0.048695,0.351014,0.109952
Human issues,0.048138,0.081885,0.085740,0.048695,0.045919,0.084525


,Empty travel of MHE flow (inter-functional),Information flow (inter-functional),Loaded travel of MHE flow (inter-functional),Loaded travel of MHE flow (intra-functional),Other equipments flow (inter-functional),Personnel flow (inter-functional),Ease of expansion,Number of free space Locations with good shape factor,Number of free space Locations with usable area,Total area of free space,Accessibility of alternate routes,Average number of alternate routes,Material handling flexibility,Variations demand volume,Variations in MHC,Non-value adding area,Value adding area,Ease of supervision and control,Safety and housekeeping
Empty travel of MHE flow (inter-functional),0.000000,0.185000,0.567000,0.000000,0.185000,0.240000,0.188000,0.000000,0.000000,0.000000,0.205000,0.205000,0.205000,0.000000,0.205000,0.205000,0.205000,0.150000,0.205000
Information flow (inter-functional),0.100000,0.000000,0.117000,0.000000,0.097000,0.085000,0.099000,0.000000,0.000000,0.000000,0.069000,0.069000,0.069000,0.000000,0.069000,0.069000,0.069000,0.088000,0.069000
Loaded travel of MHE flow (inter-functional),0.600000,0.532000,0.000000,0.000000,0.532000,0.523000,0.327000,0.000000,0.000000,0.000000,0.323000,0.323000,0.323000,0.000000,0.323000,0.323000,0.323000,0.307000,0.323000
Loaded travel of MHE flow (intra-functional),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188000,0.000000,0.000000,0.000000,0.205000,0.205000,0.205000,0.000000,0.205000,0.205000,0.205000,0.217000,0.205000
Other equipments flow (inter-functional),0.100000,0.097000,0.117000,0.000000,0.000000,0.152000,0.099000,0.000000,0.000000,0.000000,0.069000,0.069000,0.069000,0.000000,0.069000,0.069000,0.069000,0.088000,0.069000
Personnel flow (inter-functional),0.200000,0.185000,0.199000,0.000000,0.185000,0.000000,0.099000,0.000000,0.000000,0.000000,0.128000,0.128000,0.128000,0.000000,0.128000,0.128000,0.128000,0.150000,0.128000
Ease of expansion,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.400000,0.500000,0.284000,0.284000,0.284000,0.278000,0.000000,0.278000,0.278000,0.000000,0.000000
Number of free space Locations with good shape factor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333000,0.000000,0.200000,0.250000,0.099000,0.099000,0.099000,0.163000,0.000000,0.163000,0.163000,0.000000,0.000000
Number of free space Locations with usable area,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333000,0.200000,0.000000,0.250000,0.099000,0.099000,0.099000,0.163000,0.000000,0.163000,0.163000,0.000000,0.000000
Total area of free space,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333000,0.400000,0.400000,0.000000,0.518000,0.518000,0.518000,0.395000,0.000000,0.395000,0.395000,1.000000,1.000000


,Empty travel of MHE flow (inter-functional),Information flow (inter-functional),Loaded travel of MHE flow (inter-functional),Loaded travel of MHE flow (intra-functional),Other equipments flow (inter-functional),Personnel flow (inter-functional),Ease of expansion,Number of free space Locations with good shape factor,Number of free space Locations with usable area,Total area of free space,Accessibility of alternate routes,Average number of alternate routes,Material handling flexibility,Variations demand volume,Variations in MHC,Non-value adding area,Value adding area,Ease of supervision and control,Safety and housekeeping
Empty travel of MHE flow (inter-functional),0.000000,0.102987,0.315642,0.000000,0.102987,0.133605,0.014147,0.000000,0.000000,0.000000,0.044311,0.044311,0.044311,0.000000,0.058378,0.034260,0.034260,0.052739,0.072076
Information flow (inter-functional),0.055669,0.000000,0.065133,0.000000,0.053999,0.047319,0.007450,0.000000,0.000000,0.000000,0.014914,0.014914,0.014914,0.000000,0.019649,0.011531,0.011531,0.030940,0.024260
Loaded travel of MHE flow (inter-functional),0.334013,0.296158,0.000000,0.000000,0.296158,0.291148,0.024607,0.000000,0.000000,0.000000,0.069816,0.069816,0.069816,0.000000,0.091981,0.053981,0.053981,0.107939,0.113564
Loaded travel of MHE flow (intra-functional),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014147,0.000000,0.000000,0.000000,0.044311,0.044311,0.044311,0.000000,0.058378,0.034260,0.034260,0.076295,0.072076
Other equipments flow (inter-functional),0.055669,0.053999,0.065133,0.000000,0.000000,0.084617,0.007450,0.000000,0.000000,0.000000,0.014914,0.014914,0.014914,0.000000,0.019649,0.011531,0.011531,0.030940,0.024260
Personnel flow (inter-functional),0.111338,0.102987,0.110781,0.000000,0.102987,0.000000,0.007450,0.000000,0.000000,0.000000,0.027667,0.027667,0.027667,0.000000,0.036451,0.021392,0.021392,0.052739,0.045004
Ease of expansion,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125546,0.125546,0.156932,0.038432,0.038432,0.038432,0.079166,0.000000,0.070288,0.070288,0.000000,0.000000
Number of free space Locations with good shape factor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104517,0.000000,0.062773,0.078466,0.013397,0.013397,0.013397,0.046418,0.000000,0.041212,0.041212,0.000000,0.000000
Number of free space Locations with usable area,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104517,0.062773,0.000000,0.078466,0.013397,0.013397,0.013397,0.046418,0.000000,0.041212,0.041212,0.000000,0.000000
Total area of free space,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104517,0.125546,0.125546,0.000000,0.070097,0.070097,0.070097,0.112484,0.000000,0.099869,0.099869,0.150278,0.150278


,Empty travel of MHE flow (inter-functional),Information flow (inter-functional),Loaded travel of MHE flow (inter-functional),Loaded travel of MHE flow (intra-functional),Other equipments flow (inter-functional),Personnel flow (inter-functional),Ease of expansion,Number of free space Locations with good shape factor,Number of free space Locations with usable area,Total area of free space,Accessibility of alternate routes,Average number of alternate routes,Material handling flexibility,Variations demand volume,Variations in MHC,Non-value adding area,Value adding area,Ease of supervision and control,Safety and housekeeping
Empty travel of MHE flow (inter-functional),0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675,0.062675
Information flow (inter-functional),0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211,0.021211
Loaded travel of MHE flow (inter-functional),0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561,0.085561
Loaded travel of MHE flow (intra-functional),0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644,0.023644
Other equipments flow (inter-functional),0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635,0.022635
Personnel flow (inter-functional),0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426,0.037426
Ease of expansion,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379,0.049379
Number of free space Locations with good shape factor,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415
Number of free space Locations with usable area,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415,0.028415
Total area of free space,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192,0.069192


,Totals,Priorities,Empty travel of MHE flow (inter-functional),Information flow (inter-functional),Loaded travel of MHE flow (inter-functional),Loaded travel of MHE flow (intra-functional),Other equipments flow (inter-functional),Personnel flow (inter-functional),Ease of expansion,Number of free space Locations with good shape factor,Number of free space Locations with usable area,Total area of free space,Accessibility of alternate routes,Average number of alternate routes,Material handling flexibility,Variations demand volume,Variations in MHC,Non-value adding area,Value adding area,Ease of supervision and control,Safety and housekeeping
Limit priorities,nan,nan,0.062675,0.021211,0.085561,0.023644,0.022635,0.037426,0.049379,0.028415,0.028415,0.069192,0.064238,0.076101,0.113718,0.104459,0.018786,0.069000,0.069000,0.016092,0.040054
Facility 1,0.839446,0.317112,0.983936,0.812500,0.946646,1.000000,1.000000,0.883721,0.874000,1.000000,0.928571,0.920897,0.630000,1,1,0.442609,0.961000,0.913997,0.925569,0.303000,0.303000
Facility 2,0.852402,0.322007,1.000000,1.000000,1.000000,1.000000,0.942029,1.000000,0.382000,0.600000,0.833333,1.000000,0.794000,1,1,0.442609,1,0.973659,0.878730,1,0.550000
Facility 3,0.955309,0.360881,0.962672,0.714286,0.943769,0.849771,0.977444,0.644068,1,0.840000,1.000000,0.976949,1,1,1,1.000000,0.962000,1.000000,1.000000,0.550000,1
